In [1]:
# UT Bootcamp - Sections 6.5.1 to 6.5.4 
import pandas as pd
import gmaps  # for heatmaps
import requests
import pprint
from config import g_key  # Import the API key.

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,East London,ZA,2021-06-02 16:55:55,-33.0153,27.9116,61.74,88,20,11.50
1,1,Mataura,NZ,2021-06-02 16:55:57,-46.1927,168.8643,40.41,97,18,3.42
2,2,Saint-Pierre,RE,2021-06-02 16:51:02,-21.3393,55.4781,74.88,80,0,12.66
3,3,Ushuaia,AR,2021-06-02 16:54:47,-54.8000,-68.3000,33.42,93,75,4.61
4,4,Chimore,BO,2021-06-02 16:56:02,-16.9833,-65.1333,84.07,54,92,4.29


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap for Max Temp
locations = city_data_df[["Lat", "Lng"]]
max_temp = city_data_df["Max Temp"]

# Handle negative max temps by using 0 instead of negative since Google heatmaps don't plot negative numbers
temps = []
for t in max_temp: 
    temps.append(max(t, 0))
# Alternate option with list comprehension: temps = [max(temp, 0) for temp in max_temp]
# or skip defining temps and just put list comprehension in weights = _ below

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)  
heat_layer= gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [10]:
# Filter the dataset to find the cities that fit the user input criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Saint-Pierre,RE,2021-06-02 16:51:02,-21.3393,55.4781,74.88,80,0,12.66
4,4,Chimore,BO,2021-06-02 16:56:02,-16.9833,-65.1333,84.07,54,92,4.29
5,5,Hithadhoo,MV,2021-06-02 16:56:03,-0.6000,73.0833,83.73,68,71,18.05
7,7,Kavieng,PG,2021-06-02 16:56:06,-2.5744,150.7967,80.82,82,96,9.71
11,11,Hinesville,US,2021-06-02 16:55:58,31.8469,-81.5959,84.16,88,75,3.44
12,12,Whitefish,US,2021-06-02 16:55:06,48.4111,-114.3376,75.11,56,23,3.31
13,13,Rikitea,PF,2021-06-02 16:56:13,-23.1203,-134.9692,72.00,60,98,18.79
16,16,Tamworth,GB,2021-06-02 16:56:18,52.6340,-1.6959,77.32,80,96,8.08
18,18,Manokwari,ID,2021-06-02 16:52:21,-0.8667,134.0833,79.36,79,93,5.99
25,25,Lata,PT,2021-06-02 16:56:33,40.1629,-8.3327,72.72,77,98,8.75


In [11]:
preferred_cities_df.count()

City_ID       209
City          209
Country       209
Date          209
Lat           209
Lng           209
Max Temp      209
Humidity      209
Cloudiness    209
Wind Speed    209
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Saint-Pierre,RE,74.88,-21.3393,55.4781,
4,Chimore,BO,84.07,-16.9833,-65.1333,
5,Hithadhoo,MV,83.73,-0.6000,73.0833,
7,Kavieng,PG,80.82,-2.5744,150.7967,
11,Hinesville,US,84.16,31.8469,-81.5959,
12,Whitefish,US,75.11,48.4111,-114.3376,
13,Rikitea,PF,72.00,-23.1203,-134.9692,
16,Tamworth,GB,77.32,52.6340,-1.6959,
18,Manokwari,ID,79.36,-0.8667,134.0833,
25,Lata,PT,72.72,40.1629,-8.3327,


In [21]:
# # Use Google API nearby search to look for hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Parameters for hotel search
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude for each row.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f"Hotel found in {lat},{lng}")
    except (IndexError):
        print("Hotel not found... skipping")

Hotel found in -21.3393,55.4781
Hotel found in -16.9833,-65.1333
Hotel found in -0.6,73.0833
Hotel found in -2.5744,150.7967
Hotel found in 31.8469,-81.5959
Hotel found in 48.4111,-114.3376
Hotel found in -23.1203,-134.9692
Hotel found in 52.634,-1.6959
Hotel found in -0.8667,134.0833
Hotel found in 40.1629,-8.3327
Hotel found in -4.1967,152.1721
Hotel found in 42.2506,-75.833
Hotel found in 35.1333,140.3
Hotel found in 40.4272,71.7189
Hotel found in 46.3333,2.6
Hotel found in 63.8709,11.2999
Hotel found in 42.097,-79.2353
Hotel found in -24.7911,-50.0119
Hotel not found... skipping
Hotel found in -14.25,129.55
Hotel found in 57.9727,-3.978
Hotel found in 23.5,87.75
Hotel found in 11.3,106.1
Hotel found in -9.8,-139.0333
Hotel found in -0.7393,-90.3518
Hotel found in 19.7297,-155.09
Hotel found in -6.8844,112.2051
Hotel found in -3.8004,102.2655
Hotel found in 12.1791,125.5072
Hotel found in 4.8845,-1.7554
Hotel found in -20.2078,57.7625
Hotel found in 47.6666,-52.7314
Hotel found in -

In [22]:
hotel_df.head(20)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Saint-Pierre,RE,74.88,-21.3393,55.4781,Lindsey Hôtel
4,Chimore,BO,84.07,-16.9833,-65.1333,Bichito
5,Hithadhoo,MV,83.73,-0.6000,73.0833,Scoop Guest House
7,Kavieng,PG,80.82,-2.5744,150.7967,Nusa Island Retreat
11,Hinesville,US,84.16,31.8469,-81.5959,Holiday Inn Express & Suites Hinesville East -...
12,Whitefish,US,75.11,48.4111,-114.3376,Best Western Rocky Mountain Lodge
13,Rikitea,PF,72.00,-23.1203,-134.9692,Pension Maro'i
16,Tamworth,GB,77.32,52.6340,-1.6959,Premier Inn Tamworth Central hotel
18,Manokwari,ID,79.36,-0.8667,134.0833,Swiss Belhotel Manokwari
25,Lata,PT,72.72,40.1629,-8.3327,Casa Maquia


In [23]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [25]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))